In [1]:
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = "/content/drive/MyDrive/AI System"
print("已挂载，工作目录：", BASE_DIR)

Mounted at /content/drive
已挂载，工作目录： /content/drive/MyDrive/AI System


In [2]:
!pip -q install pandas pyarrow

import pandas as pd
from pathlib import Path


DATA_FILE = "mental_health_counseling_conversations_rated.csv"
DATA_PATH = Path(f"{BASE_DIR}") / DATA_FILE

print("文件路径:", DATA_PATH)

文件路径: /content/drive/MyDrive/AI System/mental_health_counseling_conversations_rated.csv


In [3]:
import pandas as pd

# 读取文件
path = "/content/drive/MyDrive/AI System/mental_health_counseling_conversations_rated.csv"
df = pd.read_csv(path)
df.head()


,index,context,response,empathy_llama-3-2-1b,empathy_llama-3-2-3b,empathy_llama-3-1-8b,empathy_qwen-2-5-7b,appropriateness_llama-3-2-1b,appropriateness_llama-3-2-3b,appropriateness_llama-3-1-8b,...,explanation_llama-3-2-3b,explanation_llama-3-1-8b,explanation_qwen-2-5-7b,generated_text_llama-3-2-1b,generated_text_llama-3-2-3b,generated_text_llama-3-1-8b,generated_text_qwen-2-5-7b,avg_empathy_score,avg_appropriateness_score,avg_relevance_score
0,0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb...",2,4,2,2,1,3,4,...,The response acknowledges the user's feelings ...,The response acknowledges the user's feelings ...,The response minimally acknowledges the user's...,Empathy: 2 \nAppropriateness: 1 \nRelevance:...,Empathy: 4 \nAppropriateness: 3 \nRelevance:...,Empathy: 2 \nAppropriateness: 4 \nRelevance:...,Empathy: 2 \nAppropriateness: 3 \nRelevance:...,2.50,2.75,4.25
1,1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see...",5,5,4,4,5,5,5,...,The response demonstrates strong empathy by ac...,The response demonstrates good empathy by ackn...,The response shows good empathy by acknowledgi...,Empathy: 5 \nAppropriateness: 5 \nRelevance:...,Empathy: 5 \nAppropriateness: 5 \nRelevance:...,Empathy: 4 \nAppropriateness: 5 \nRelevance:...,Empathy: 4 \nAppropriateness: 5 \nRelevance:...,4.50,5.00,4.75
2,2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...,2,4,3,3,4,5,4,...,The response acknowledges the user's emotional...,The response shows moderate empathy by acknowl...,The response shows moderate empathy by acknowl...,Empathy: 2 \nAppropriateness: 4 \nRelevance:...,Empathy: 4 \nAppropriateness: 5 \nRelevance:...,Empathy: 3 \nAppropriateness: 4 \nRelevance:...,Empathy: 3 \nAppropriateness: 4 \nRelevance:...,3.00,4.25,3.25
3,3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...,3,4,2,3,5,5,4,...,The response acknowledges the user's emotional...,The response provides a relevant and informati...,The response shows moderate empathy by acknowl...,Empathy: 3 \nAppropriateness: 5 \nRelevance:...,Empathy: 4 \nAppropriateness: 5 \nRelevance:...,Empathy: 2 \nAppropriateness: 4 \nRelevance:...,Empathy: 3 \nAppropriateness: 4 \nRelevance:...,3.00,4.50,4.75
4,4,I'm going through some things with my feelings...,I first want to let you know that you are not ...,2,4,4,3,5,5,4,...,The response acknowledges the user's feelings ...,The response acknowledges the user's feelings ...,The response shows moderate empathy by acknowl...,Empathy: 2 \nAppropriateness: 5 \nRelevance:...,Empathy: 4 \nAppropriateness: 5 \nRelevance:...,Empathy: 4 \nAppropriateness: 4 \nRelevance:...,Empathy: 3 \nAppropriateness: 4 \nRelevance:...,3.25,4.50,3.75


In [4]:
import re

EMAIL_RE = r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"
PHONE_RE = r"\b(\+?\d{1,3}[-.\s]?)?(\(?\d{3}\)?[-.\s]?)\d{3}[-.\s]?\d{4}\b"
DISCORD_RE = r"[A-Za-z0-9_.]{2,32}#[0-9]{4}"

def scan_privacy(df, cols):
    findings = []
    for col in cols:
        for i, text in enumerate(df[col].astype(str)):
            flags = []
            if re.search(EMAIL_RE, text): flags.append("email")
            if re.search(PHONE_RE, text): flags.append("phone")
            if re.search(DISCORD_RE, text): flags.append("contact")
            if any(k in text.lower() for k in ["diagnosed", "bipolar", "depression", "adhd", "抑郁", "精神病"]):
                flags.append("medical_term")
            if flags:
                findings.append({
                    "row": i,
                    "column": col,
                    "flags": flags,
                    "text_snippet": text[:120]
                })
    return pd.DataFrame(findings)

privacy_flags = scan_privacy(df, ["context", "response"])
print(f"Found {len(privacy_flags)} possible privacy risks")
privacy_flags.head()



Found 678 possible privacy risks


,row,column,flags,text_snippet
0,23,context,[medical_term],I have so many issues to address. I have a his...
1,24,context,[medical_term],I have so many issues to address. I have a his...
2,25,context,[medical_term],I have so many issues to address. I have a his...
3,26,context,[medical_term],I have so many issues to address. I have a his...
4,27,context,[medical_term],I have so many issues to address. I have a his...


In [5]:
import re

HIGH_RISK_PATTERNS = [
    "i want to die", "i don't want to live", "kill myself",
    "hurt myself", "end it all", "took pills", "suicide",
    "i'm worthless", "hopeless", "i will die",
    "我不想活了", "我想自杀", "我打算结束", "我要死了"
]

def scan_crisis(df, text_cols):
    alerts = []
    for col in text_cols:
        for i, text in enumerate(df[col].astype(str).str.lower()):
            hits = [p for p in HIGH_RISK_PATTERNS if p in text]
            if hits:
                alerts.append({
                    "row": i,
                    "column": col,
                    "triggers": hits,
                    "snippet": text[:120]
                })
    return pd.DataFrame(alerts)

crisis_flags = scan_crisis(df, ["context", "response"])
print(f"Found {len(crisis_flags)} potential high-risk sentences")
crisis_flags.head()

Found 189 potential high-risk sentences


,row,column,triggers,snippet
0,0,context,"[suicide, i'm worthless]",i'm going through some things with my feelings...
1,1,context,"[suicide, i'm worthless]",i'm going through some things with my feelings...
2,2,context,"[suicide, i'm worthless]",i'm going through some things with my feelings...
3,3,context,"[suicide, i'm worthless]",i'm going through some things with my feelings...
4,4,context,"[suicide, i'm worthless]",i'm going through some things with my feelings...


In [6]:
summary = {
    "total_records": len(df),
    "privacy_risks": len(privacy_flags),
    "crisis_risks": len(crisis_flags),
    "privacy_risk_ratio": round(len(privacy_flags) / len(df), 4),
    "crisis_risk_ratio": round(len(crisis_flags) / len(df), 4),
}
pd.DataFrame([summary])

,total_records,privacy_risks,crisis_risks,privacy_risk_ratio,crisis_risk_ratio
0,3512,678,189,0.1931,0.0538


In [7]:
privacy_flags.to_csv("/content/drive/MyDrive/AI System/privacy_flags.csv", index=False)
crisis_flags.to_csv("/content/drive/MyDrive/AI System/crisis_flags.csv", index=False)
df.to_csv("/content/drive/MyDrive/AI System/cleaned_data.csv", index=False)

print("All results saved in Colab /content folder.")

All results saved in Colab /content folder.


In [8]:
import pandas as pd
import re
import json
from datetime import datetime
import os

# ========== 0. basic setting ==========
RAW_CSV_PATH = "/content/drive/MyDrive/AI System/mental_health_counseling_conversations_rated.csv"

CLEAN_CSV_PATH = "/content/drive/MyDrive/AI System/cleaned_data.csv"
PRIVACY_FLAGS_PATH = "/content/drive/MyDrive/AI System/privacy_flags.csv"
CRISIS_FLAGS_PATH = "/content/drive/MyDrive/AI System/crisis_flags.csv"
RISK_LOG_PATH = "/content/drive/MyDrive/AI System/risk_log.jsonl"
RISK_SUMMARY_PATH = "/content/drive/MyDrive/AI System/risk_summary.json"

TEXT_COLS = ["context", "response"]

# 确保日志文件存在（如果不存在就创建空文件）
if not os.path.exists(RISK_LOG_PATH):
    with open(RISK_LOG_PATH, "w", encoding="utf-8") as f:
        pass

# 小工具：往风险日志里追加一条记录
def log_risk_event(event_type, payload_dict):
    record = {
        "time": datetime.utcnow().isoformat() + "Z",
        "event_type": event_type,
        "payload": payload_dict
    }
    with open(RISK_LOG_PATH, "a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")


# ========== 1. read-in data ==========
df = pd.read_csv(RAW_CSV_PATH)

# 防御式检查一下列是否存在
for col in TEXT_COLS:
    if col not in df.columns:
        raise ValueError(f"列 {col} 不在CSV里，请检查列名。实际列有：{df.columns.tolist()}")

log_risk_event("load_dataset", {
    "rows": len(df),
    "cols": list(df.columns)
})

print("数据读取完成，行数:", len(df))


# ========== 2. Define the scanning rules ==========

EMAIL_RE = r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"
PHONE_RE = r"\b(\+?\d{1,3}[-.\s]?)?(\(?\d{3}\)?[-.\s]?)\d{3}[-.\s]?\d{4}\b"
DISCORD_RE = r"[A-Za-z0-9_.]{2,32}#[0-9]{4}"

HIGH_RISK_PATTERNS = [
    "i want to die",
    "i don't want to live",
    "kill myself",
    "end it all",
    "i took pills",
    "i hurt myself",
    "suicide",
    "i will die",
    "i am worthless",
    "i want to hurt someone",
]

MENTAL_HEALTH_TERMS = [
    "diagnosed", "diagnosis", "bipolar", "schizophrenia", "adhd",
    "depression", "anxiety", "panic attack", "ptsd",
    "Depression "," anxiety ", "mental illness "," schizophrenia ", "Bipolar disorder "," bipolar affective Disorder"
]


# ========== 3.Privacy & Crisis Scan  ==========

privacy_findings = []
crisis_findings = []

for col in TEXT_COLS:
    for i, raw_text in enumerate(df[col].astype(str)):
        text_lower = raw_text.lower()

        # --- 隐私/敏感信息检测 ---
        flags = []
        if re.search(EMAIL_RE, raw_text):
            flags.append("email")
        if re.search(PHONE_RE, raw_text):
            flags.append("phone")
        if re.search(DISCORD_RE, raw_text):
            flags.append("contact_handle")
        if any(term in text_lower for term in MENTAL_HEALTH_TERMS):
            flags.append("medical/mental_health_ref")

        if flags:
            finding = {
                "row": int(i),
                "column": col,
                "flags": flags,
                "text_snippet": raw_text[:160]
            }
            privacy_findings.append(finding)
            log_risk_event("privacy_flag", finding)

        # --- 危机/高风险内容检测 ---
        hits = [p for p in HIGH_RISK_PATTERNS if p in text_lower]
        if hits:
            crisis_item = {
                "row": int(i),
                "column": col,
                "triggers": hits,
                "text_snippet": raw_text[:200]
            }
            crisis_findings.append(crisis_item)
            log_risk_event("crisis_flag", crisis_item)

print(f"privacy items: {len(privacy_findings)}")
print(f"crisis items: {len(crisis_findings)}")


privacy_flags_df = pd.DataFrame(privacy_findings)
crisis_flags_df = pd.DataFrame(crisis_findings)

# 存储扫描明细
privacy_flags_df.to_csv(PRIVACY_FLAGS_PATH, index=False)
crisis_flags_df.to_csv(CRISIS_FLAGS_PATH, index=False)


# ========== 4. 文本脱敏（anonymization） ==========

def anonymize_text(text):
    """把原始文本里的个人识别信息/联系方式打码，不改变其情绪信号。"""
    text = re.sub(EMAIL_RE, "[EMAIL]", text)
    text = re.sub(PHONE_RE, "[PHONE]", text)
    text = re.sub(DISCORD_RE, "[HANDLE]", text)
    return text

df["context_clean"] = df["context"].astype(str).apply(anonymize_text)
df["response_clean"] = df["response"].astype(str).apply(anonymize_text)

log_risk_event("anonymization_done", {
    "new_columns": ["context_clean", "response_clean"]
})

print("has generated the desensitized column context_clean / response_clean")


# ========== 5. risk_summary.json ==========

summary = {
    "total_records": len(df),
    "privacy_risk_count": len(privacy_findings),
    "crisis_risk_count": len(crisis_findings),
    "privacy_risk_ratio": round(len(privacy_findings) / max(len(df),1), 4),
    "crisis_risk_ratio": round(len(crisis_findings) / max(len(df),1), 4),
    "notes": {
        "privacy_risk_meaning": "It may containImmediate self-harm/other-harm/desperate language may occur. A security upgrade strategy is needed instead of a casual casual reply. sensitive information such as email addresses, phone numbers, contact details, and descriptions of mental health diagnoses.",
        "crisis_risk_meaning": "Immediate self-harm/other-harm/desperate language may occur. A security upgrade strategy is needed instead of a casual casual reply."
    }
}

with open(RISK_SUMMARY_PATH, "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)

log_risk_event("summary_generated", summary)

print("risk summary：")
print(json.dumps(summary, indent=2, ensure_ascii=False))


# ========== 6. 导出清洗后训练数据版本 ==========

# 我们保留原始评分列等，但强制下游训练使用 *_clean 的文本字段
export_cols = df.columns.tolist()  # 先拿所有列


df.to_csv(CLEAN_CSV_PATH, index=False)

print("prompted finished ：")
print(" - clean data :", CLEAN_CSV_PATH)
print(" - privacy data :", PRIVACY_FLAGS_PATH)
print(" - crisis :", CRISIS_FLAGS_PATH)
print(" - risk diary :", RISK_LOG_PATH)
print(" - risk summary :", RISK_SUMMARY_PATH)

/tmp/ipython-input-2344634657.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "time": datetime.utcnow().isoformat() + "Z",


数据读取完成，行数: 3512


/tmp/ipython-input-2344634657.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "time": datetime.utcnow().isoformat() + "Z",
/tmp/ipython-input-2344634657.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "time": datetime.utcnow().isoformat() + "Z",


privacy items: 1211
crisis items: 101


/tmp/ipython-input-2344634657.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "time": datetime.utcnow().isoformat() + "Z",


has generated the desensitized column context_clean / response_clean
risk summary：
{
  "total_records": 3512,
  "privacy_risk_count": 1211,
  "crisis_risk_count": 101,
  "privacy_risk_ratio": 0.3448,
  "crisis_risk_ratio": 0.0288,
  "notes": {
    "privacy_risk_meaning": "It may containImmediate self-harm/other-harm/desperate language may occur. A security upgrade strategy is needed instead of a casual casual reply. sensitive information such as email addresses, phone numbers, contact details, and descriptions of mental health diagnoses.",
    "crisis_risk_meaning": "Immediate self-harm/other-harm/desperate language may occur. A security upgrade strategy is needed instead of a casual casual reply."
  }
}
prompted finished ：
 - clean data : /content/drive/MyDrive/AI System/cleaned_data.csv
 - privacy data : /content/drive/MyDrive/AI System/privacy_flags.csv
 - crisis : /content/drive/MyDrive/AI System/crisis_flags.csv
 - risk diary : /content/drive/MyDrive/AI System/risk_log.jsonl
 - ri